In [2]:
import nltk
import numpy as np
from numpy import array, ones, zeros, multiply
import sys
from ipywidgets import FloatProgress
from IPython.display import display
import cPickle as pickle
import math

train_10 = pickle.load(open('/Users/xchen/Documents/AIC/TC4/projet/typos-data/train10.pkl', 'rb'))
train_20 = pickle.load(open('/Users/xchen/Documents/AIC/TC4/projet/typos-data/train20.pkl', 'rb'))
test_10 = pickle.load(open('/Users/xchen/Documents/AIC/TC4/projet/typos-data/test10.pkl', 'rb'))
test_20 = pickle.load(open('/Users/xchen/Documents/AIC/TC4/projet/typos-data/test20.pkl', 'rb'))
print len(train_10),len(train_20),len(test_10),len(test_20)

29057 27184 1501 3374


In [3]:
class HMM2:
        def __init__(self,state_list, observation_list, state2_list,
                 transition_proba = None,
                 transition2_proba = None,
                 observation_proba = None,
                 observation2_proba = None,
                 initial_state_proba = None,smoothing_obs = 0.01,smoothing_trans2= 0.01):
            """Builds a new Hidden Markov Model
            state_list is the list of state symbols [q_0...q_(N-1)]
            observation_list is the list of observation symbols [v_0...v_(M-1)]
            transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print "HMM creating with: "
            self.N = len(state_list) # The number of states
            self.M = len(observation_list) # The number of words in the vocabulary
            self.L = len(state2_list)
            print str(self.N)+" states"
            print str(self.L)+" joint states"
            print str(self.M)+" observations"
            self.omega_Y = state_list # Keep the vocabulary of tags
            self.omega_X = observation_list # Keep the vocabulary of tags
            self.omega_Y2 = state2_list
            # Init. of the 3 distributions : observation, transition and initial states
            if transition_proba is None:
                self.transition_proba = zeros( (self.N, self.N), float) 
            else:
                self.transition_proba=transition_proba
            if transition2_proba is None:
                self.transition2_proba = zeros((self.N,self.N,self.N),float)
            else:
                self.transition2_proba = transition2_proba
            if observation_proba is None:
                self.observation_proba = zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if observation2_proba is None:
                self.observation2_proba = zeros((self.N,self.N,self.N), float) 
            else:
                self.observation2_proba=observation2_proba
            if initial_state_proba is None:
                self.initial_state_proba = zeros( (self.N,), float ) 
            else:
                self.initial_state_proba=initial_state_proba
            # Since everything will be stored in numpy arrays, it is more convenient and compact to 
            # handle words and tags as indices (integer) for a direct access. However, we also need 
            # to keep the mapping between strings (word or tag) and indices. 
            self.make_indexes()
            self.smoothing_obs = smoothing_obs
            self.smoothing_trans2 = smoothing_trans2

        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities arrays"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
            self.Y2_index = {}
            k=0
            for i in self.omega_Y:
                for j in self.omega_Y:
                    self.Y2_index[(j,i)] =k
                    k+=1
                
        def get_observationIndices( self, observations ):
            """return observation indices, i.e 
            return [self.O_index[o] for o in observations]
            and deals with OOVs
            """
            indices = zeros( len(observations), int )
            k = 0
            for o in observations:
                if o in self.X_index:
                    indices[k] = self.X_index[o]
                else:
                    indices[k] = UNKid
                k += 1
            return indices
        
        def data2indices(self, sent): 
            """From one tagged sentence of the brown corpus: 
            - extract the words and tags 
            - returns two list of indices, one for each
            -> (wordids, tagids)
            """
            wordids = list()
            tagids  = list()
            for couple in sent:
                wrd = couple[0]
                tag = couple[1]
                if wrd in self.X_index:
                    wordids.append(self.X_index[wrd])
                else:
                    wordids.append(UNKid)
                tagids.append(self.Y_index[tag])
            return wordids,tagids
        
        def observation_estimation(self,pair_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill in with counts
            for pair in pair_counts:
                wrd=pair[0]
                tag=pair[1]
                cpt=pair_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i=self.Y_index[tag]
                self.observation_proba[k,i]=cpt
            # normalize
            self.observation_proba=self.observation_proba+self.smoothing_obs
            self.observation_proba=self.observation_proba/self.observation_proba.sum(axis=0).reshape(1,self.N)
            
        def observation2_estimation(self,pair2_counts):
            """ Build the observation distribution: 
                observation_proba is the observation probablility matrix
                    [b_ki],  b_ki = Pr(X_t=v_k|Y_t=q_i)"""
            # fill in with counts
            for pair in pair2_counts:
                tag1=pair[0][0]
                tag2=pair[0][1]
                wrd=pair[1]
                cpt=pair2_counts[pair]
                k = 0 # for <unk>
                if wrd in self.X_index: 
                    k=self.X_index[wrd]
                i=self.Y_index[tag1]
                j=self.Y_index[tag2]
                self.observation2_proba[k,i,j]=cpt
            # normalize
            self.observation2_proba=self.observation2_proba+self.smoothing_obs
            self.observation2_proba=self.observation2_proba/self.observation2_proba.sum(axis=0).reshape(1,self.N,self.N)
        
        def transition_estimation(self,trans_counts):
            """ Build the transition distribution: 
                transition_proba is the transition matrix with : 
                [a_ij] a[i,j] = Pr(Y_(t+1)=q_i|Y_t=q_j)
            """
            # fill with counts
            for trans in trans_counts:
                i=self.Y_index[trans[0]]
                j=self.Y_index[trans[1]]
                cpt = trans_counts[trans]
                self.transition_proba[j,i] = cpt
            # normalize
            self.transition_proba = self.transition_proba/self.transition_proba.sum(axis =0).reshape(1,self.N)
                
        def transition2_estimation(self,trans2_counts):
            for trans in trans2_counts:
                i = self.Y_index[trans[0][0]]
                j = self.Y_index[trans[0][1]]
                k = self.Y_index[trans[1]]
                cpt = trans2_counts[trans]
                self.transition2_proba[k,i,j] = cpt
            # normalize
            self.transition2_proba=self.transition2_proba+self.smoothing_trans2
            self.transition2_proba = self.transition2_proba/self.transition2_proba.sum(axis=0).reshape(1,self.N,self.N)
    
        def init_estimation(self,init_counts):
            for init in init_counts:
                index = self.Y_index[init]
                self.initial_state_proba[index] = init_counts[init]
            self.initial_state_proba = self.initial_state_proba/sum(self.initial_state_proba)
            
        def trans2_index(self):
            self.Y2_state_index = {}
            for i in self.Y2_index:
                index = self.Y2_index[i]
                i_1 = self.Y_index[i[1]]
                self.Y2_state_index[index] = i_1
        
        def supervised_training(self, pair_counts, pair2_counts,trans_counts,trans2_counts,init_counts):
            """ Train the HMM's parameters. This function wraps everything"""
            self.trans2_index()
            self.observation_estimation(pair_counts)
            self.transition_estimation(trans_counts)
            self.transition2_estimation(trans2_counts)
            self.init_estimation(init_counts)
            self.observation2_estimation(pair2_counts)
            
        def viterbi(self,obs): 
            B = self.observation_proba
            B2 = self.observation2_proba
            A = self.transition_proba
            A2 = self.transition2_proba
            T = len(obs)
            N = self.N
            
            
            delta = zeros((N,N),float)
            psi = zeros((T-1,N,N),int)
            delta_t = zeros((N,N),float)
            
            
            if T<=2:
                return self.viterbi1(obs)
            else:
                init = B[obs[0]]*self.initial_state_proba
                for i in xrange(N):
                    for j in xrange(N):
                        delta[i,j]=init[i]*A[j,i]*B[obs[1]][j]
                
                for t in xrange(2,T):
                    O_t = obs[t]
                    for j in range(N):
                        for k in range(N):
                            temp = []
                            for i in range(N):
                                tmp = multiply(delta[i,j],A2[k,i,j])
                                temp.append(tmp)
                            idx = psi[t-1,j,k] = np.asarray(temp).argmax()
                            delta_t[j,k] = temp[idx]*B2[O_t,j,k]
                    delta,delta_t = delta_t,delta 
            
                max_index = delta.argmax()
                i_star = [max_index%N]
                i_star.append(max_index/N)
                temp = (i_star[1],i_star[0])
            
                for psi_t in psi[T-1:0:-1]:
                    i_star.append(psi_t[temp[0]][temp[1]])
                    temp = (psi_t[temp[0]][temp[1]],temp[0])
                i_star.reverse()
                return i_star
            
        def viterbi1(self,obs):
            B = self.observation_proba
            A = self.transition_proba
            T = len(obs)
            N = self.N
            
            delta = zeros(N,float)
            tmp = zeros(N,float)
            psi = zeros((T,N),int)
            delta_t = zeros(N,float)
            
            delta = B[obs[0]]*self.initial_state_proba
            for t in xrange(1,T):
                O_t = obs[t]
                for j in range(N):
                    tmp = multiply(delta,A[j,:])
                    idx = psi[t,j] = tmp.argmax()
                    delta_t[j] = tmp[idx]*B[O_t,j]
                delta,delta_t = delta_t,delta
            i_star = [delta.argmax()]
            temp = delta.argmax()
            for psi_t in psi[T-1:0:-1]:
                i_star.append(psi_t[temp])
                temp = psi_t[temp]
            i_star.reverse()
            return i_star

In [4]:
def make_counts2(data):
    c_word = {}
    c_tag = {}
    c_tag2 = {}
    c_pairs = {}
    c_pairs2 = {}
    c_transition = {}
    c_transition2 = {}
    c_init = {}
    for d in data:
        if not c_init.has_key(d[0][1]):
            c_init[d[0][1]] = 1
        else: c_init[d[0][1]] = c_init.get(d[0][1])+1
        for i in xrange(len(d)):
            if not c_word.has_key(d[i][0]):
                c_word[d[i][0]] = 1
            else: c_word[d[i][0]] = c_word.get(d[i][0])+1
            if not c_tag.has_key(d[i][1]):
                c_tag[d[i][1]] = 1
            else: c_tag[d[i][1]] = c_tag.get(d[i][1])+1
            if not c_pairs.has_key(d[i]):
                c_pairs[d[i]] = 1
            else: c_pairs[d[i]] = c_pairs.get(d[i])+1
            if i <= len(d)-2:
                if not c_pairs2.has_key(((d[i][1],d[i+1][1]),d[i+1][0])):
                    c_pairs2[((d[i][1],d[i+1][1]),d[i+1][0])]=1
                else: c_pairs2[((d[i][1],d[i+1][1]),d[i+1][0])] = c_pairs2.get(((d[i][1],d[i+1][1]),d[i+1][0]))+1
            if i <= len(d)-2:
                if not c_transition.has_key((d[i][1],d[i+1][1])):
                    c_transition[(d[i][1],d[i+1][1])] = 1
                else: c_transition[(d[i][1],d[i+1][1])] = c_transition.get((d[i][1],d[i+1][1]))+1
            if i <= len(d)-2:
                if not c_tag2.has_key((d[i][1],d[i+1][1])):
                    c_tag2[(d[i][1],d[i+1][1])]=1
                else: c_tag2[(d[i][1],d[i+1][1])] = c_tag2.get((d[i][1],d[i+1][1]))+1
            if i <= len(d)-3:
                if not c_transition2.has_key(((d[i][1],d[i+1][1]),d[i+2][1])):
                    c_transition2[((d[i][1],d[i+1][1]),d[i+2][1])] = 1
                else: c_transition2[((d[i][1],d[i+1][1]),d[i+2][1])] = c_transition2.get(((d[i][1],d[i+1][1]),d[i+2][1]))+1 
    
    return c_word,c_tag,c_tag2,c_pairs,c_pairs2,c_transition,c_transition2,c_init

In [8]:
c_words_train2,c_tags_train2,c_tags2_train2,c_pairs_train2,c_pairs2_train2,c_transition_train2,c_transition2_train2,c_init_train = make_counts2(train_10)
print'number of observations:',len(c_words_train2)
print 'number of states:',len(c_tags_train2)
print 'number of obs/state pairs:',len(c_pairs_train2)
print 'number of state-transition(first order):',len(c_transition_train2)
print 'number of initial state:',len(c_init_train)
print 'number of joint-states:',len(c_tags2_train2)
print 'number of obs2/(state1,state2):',len(c_pairs2_train2)
print 'number of state-transition(second order):',len(c_transition2_train2)

number of observations: 26
number of states: 26
number of obs/state pairs: 127
number of state-transition(first order): 403
number of initial state: 25
number of joint-states: 403
number of obs2/(state1,state2): 1490
number of state-transition(second order): 2489


In [68]:
hmm2_train = HMM2(state_list=c_tags_train2.keys(), observation_list=c_words_train2.keys(),
                 state2_list= c_tags2_train2.keys(),
                 transition_proba = None,
                 observation_proba = None,
                 initial_state_proba = None)
hmm2_train.supervised_training(c_pairs_train2,c_pairs2_train2,c_transition_train2,c_transition2_train2,c_init_train)
print hmm2_train.observation_proba.sum(axis =0)
print hmm2_train.transition_proba.sum(axis =0)
print hmm2_train.transition2_proba.sum(axis=0)
print hmm2_train.observation2_proba.sum(axis=0)
print sum(hmm2_train.initial_state_proba)
print len(hmm2_train.observation_proba)
print len(hmm2_train.transition_proba)
print len(hmm2_train.transition2_proba)
print len(hmm2_train.initial_state_proba)

HMM creating with: 
26 states
403 joint states
26 observations
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.

In [69]:
tot_2 = 0.0
correct_2 = 0.0
confusion_2 = zeros([hmm2_train.N,hmm2_train.N])
f = FloatProgress(min=0, max=len(test_10))
display(f)
for test in test_10:
    f.value+=1
    wordids_test,tagids_test = hmm2_train.data2indices(test)
    tagids_pre = hmm2_train.viterbi(wordids_test)
    for i in xrange(len(tagids_pre)):
        hyp = tagids_pre[i]
        ref = tagids_test[i]
        if hyp == ref:
            correct_2+=1
        confusion_2[hyp][ref]+=1
        tot_2+=1
print "OK : "+str(correct_2)+" / "+str(tot_2)+ " -> "+ str(correct_2*100/tot_2)

OK : 6982.0 / 7320.0 -> 95.3825136612
